In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import strip_tags
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

//anaconda/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
file = 'yelp_data_official_training.csv'
df = pd.DataFrame(columns=['id', 'category', 'review'])
i = 0
with open(file, encoding="utf8") as f:
    f.readline()
    for line in f:
        line = line.rstrip('\n').split('|')
        if len(line) > 1:
            if i != 0:
                one_record['review'] = review
                #print(one_record)
                df = df.append(one_record, ignore_index=True)
            if line[0] == str(i):
                one_record = {}
                one_record['id'] = int(line[0])
                one_record['category'] = int(line[1])
                review = line[2]
            i += 1
            
        else:
            review += ' ' + line[0]           
    one_record['review'] = review
    df = df.append(one_record, ignore_index=True)
print ('train data done')

train data done


In [3]:
def make_xy(reviews, vectorizer=False, if_y=True):
    if not vectorizer:
        vectorizer = CountVectorizer() #min_df=5, stop_words='english', max_df=0.2
        vectorizer.fit(reviews['review'].tolist())
    x = vectorizer.transform(reviews['review'].tolist())
    if if_y:
        Y = np.asarray(reviews['category'])
        return x, Y, vectorizer
    else:
        return x, vectorizer

X, Y, vect = make_xy(df)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

In [4]:
vectorizer = CountVectorizer(binary=False, min_df=5, stop_words='english', max_df=0.2) 
vectorizer.fit(df['review'].tolist())
X, Y, v_new = make_xy(df, vectorizer)
# logreg = LogisticRegression(C=0.1, intercept_scaling=1, dual=False, fit_intercept=True,
#           penalty='l2', tol=0.0001)
fitted_model = MultinomialNB()
fitted_model.fit(X, Y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [5]:
score = cross_val_score(fitted_model, X, Y, scoring='accuracy').mean()

In [6]:
print('accuracy: ' + str(score))

accuracy: 0.880918930219


In [10]:
import nltk
from nltk.stem.porter import PorterStemmer

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        stems.append(PorterStemmer().stem(item))
    return stems


tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',sublinear_tf = True)

In [11]:
X = tfidf.fit_transform(df['review'].tolist())

In [12]:
logregg = LogisticRegression(C=1.0, class_weight='balanced', dual=False, fit_intercept=False,
intercept_scaling=1, max_iter=100, multi_class='ovr',
penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
verbose=0)

In [13]:
logregg.fit(X,Y)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=False, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [14]:
score = cross_val_score(logregg, X, Y, scoring='accuracy').mean()

In [15]:
print('accuracy: ' + str(score))

accuracy: 0.882210500546


In [16]:
file_test = 'yelp_data_official_test_nocategories.csv'
df_test = pd.DataFrame(columns=['id', 'review'])
i = 0
with open(file_test) as f:
    f.readline()
    for line in f:
        line = line.rstrip('\n').split('|')
        if len(line) > 1:
            if i != 0:
                one_record['review'] = review
                #print(one_record)
                df_test = df_test.append(one_record, ignore_index=True)
            if line[0] == str(i):
                one_record = {}
                one_record['id'] = int(line[0])
                review = line[1]
            i += 1
            
        else:
            review += ' ' + line[0]           
    one_record['review'] = review
    df_test = df_test.append(one_record, ignore_index=True)
print ('test data done')

test data done


In [17]:
def write_result(x, y, model, vect, version):
    test_x, v = make_xy(df_test, vect, False)
    model.fit(x, y)
    y_predict = model.predict(test_x)
    result = pd.DataFrame({'id': range(len(y_predict)), 'category': y_predict})

    result[['id', 'category']] = result[['id', 'category']].astype(int)
    result[['id', 'category']].to_csv('yelp_data_official_test_submission_' + version + '.csv', index=False)

In [18]:
write_result(X, Y, logregg, tfidf, 'v15')